![RP Tango](./dataset/Tango_Management_logo.png "RP Tango")

***
# Coronavirus COVID-19 Forecast
### Predicting Confirmed and Death cases

**Last Update : 2/17/2020** <br>
***

In [56]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import plotly.graph_objects as go 
import seaborn as sns
import plotly
import plotly.express as px
from fbprophet.plot import plot_plotly
from fbprophet import Prophet
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot, plot_mpl
import plotly.offline as py
init_notebook_mode(connected=True)

## 1. Data Exploration/Preparation

In [68]:
dataset = pd.read_csv('./dataset/2019_nCoV_data.csv')
dataset.tail()

,Sno,Date,Province/State,Country,Last Update,Confirmed,Deaths,Recovered
1564,1565,02/15/2020 22:00:00,"Madison, WI",US,2020-02-05T21:53:02,1.0,0.0,0.0
1565,1566,02/15/2020 22:00:00,"Orange, CA",US,2020-02-01T19:53:03,1.0,0.0,0.0
1566,1567,02/15/2020 22:00:00,"San Antonio, TX",US,2020-02-13T18:53:02,1.0,0.0,0.0
1567,1568,02/15/2020 22:00:00,"Seattle, WA",US,2020-02-09T07:03:04,1.0,0.0,1.0
1568,1569,02/15/2020 22:00:00,"Tempe, AZ",US,2020-02-01T19:43:03,1.0,0.0,0.0


In [70]:
dataset.isnull().sum()

Sno                 0
Date                0
Province/State    418
Country             0
Last Update         0
Confirmed           0
Deaths              0
Recovered           0
dtype: int64

In [73]:
print("# of rows in the dataset: {}".format(dataset.shape[0]))
print("# of Columns in the dataset: {}".format(dataset.shape[1]))

# of rows in the dataset: 1569
# of Columns in the dataset: 8


## 2. Training Dataset Separation

In [21]:
confirmed_training_dataset = pd.DataFrame(dataset.groupby('Date')['Confirmed'].sum().reset_index()).rename(columns={'Date': 'ds', 'Confirmed': 'y'})
death_training_dataset = pd.DataFrame(dataset.groupby('Date')['Deaths'].sum().reset_index()).rename(columns={'Date': 'ds', 'Deaths': 'y'})

## 3. Model/Prophet  setup

In [53]:
# Prophet Model for Confirmed Cases
prophetc = Prophet()
prophetc.fit(confirmed_training_dataset)
futurec = prophetc.make_future_dataframe(periods=7)
confirmed_forecast = prophetc.predict(futurec)

# Prophet Model for Death Cases
prophetd = Prophet(
    changepoint_range=0.90,
    changepoint_prior_scale=20,
    n_changepoints=17,
    yearly_seasonality=False,
    weekly_seasonality = False,
    daily_seasonality = False,
    seasonality_mode = 'additive')
prophetd.fit(death_training_dataset)
futured = prophetd.make_future_dataframe(periods=7)
deaths_forecast = prophetd.predict(futured)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 19.


## 4. Confirmed & Death Cases Visualisation

In [81]:
fig = plot_plotly(prophetc, confirmed_forecast)  
annotations = []
annotations.append(dict(xref='paper', yref='paper', x=0, y=1.15,
                              xanchor='left', yanchor='bottom',
                              text='Predictions of Confirmed Cases',
                              font=dict(family='Arial', size=22,color='rgb(37,37,37)'),showarrow=False))
fig.update_layout(annotations=annotations)
fig.show()
py.plot(fig, filename='confirmed_forecast.html')

'confirmed_forecast.html'

In [80]:
fig = plot_plotly(prophetd, deaths_forecast)  
annotations = []
annotations.append(dict(xref='paper', yref='paper', x=0, y=1.25,
                              xanchor='left', yanchor='top',
                              text='Predictions of Deaths',
                              font=dict(family='Arial', size=22,color='rgb(37,37,37)'),showarrow=False))
fig.update_layout(annotations=annotations)
fig.show()
py.plot(fig, filename='death_forcasting.html')

'death_forcasting.html'

## 5. Model Accuracy.

In [77]:
#Mean Absolute Percentage Error.
def mape(y_true, y_pred): 
    """The mean absolute percentage error (MAPE), 
    also known as mean absolute percentage deviation (MAPD), 
    is a measure of prediction accuracy of a forecasting method in statistics"""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [78]:
y_true_death = df_prophet.y.values
y_pred_daily = fcst_daily.loc[fcst_daily['ds'] <= max_date].yhat.values
y_pred_no_daily = fcst_no_daily.loc[fcst_no_daily['ds'] <= max_date].yhat.values

NameError: name 'df_prophet' is not defined